### Гипотеза

Посмотреть как влияет рассматриваемые временные окна неделя, 2 недели и месяц

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:85% !important; }</style>"))
display(HTML("<style>.prompt { min-width:10ex !important; }</style>"))
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.float_format', lambda x: '%.6f' % x) 

/tmp/ipykernel_25936/437858164.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
/tmp/ipykernel_25936/437858164.py:1: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


In [3]:
df_train = pd.read_parquet('../data/train_dataset_hackaton2023_train.gzip')
df_test = pd.read_parquet('../data/hackaton2023_test.gzip')

In [4]:
df_train.shape[0]

12129384

In [5]:
ids_cols = ['customer_id', 'buy_post', 'startdatetime']

In [6]:
df_train.describe()

,customer_id,date_diff_post,buy_post,revenue,startdatetime,ownareaall_sqm
count,12129384.000000,9660867.000000,12129384.000000,12129384.000000,12129384,12129384.000000
mean,19809542.263361,20.480386,0.796485,106.799476,2023-02-20 12:19:13.394691,215.513003
min,29891.000000,0.000000,0.000000,0.010000,2022-09-02 00:00:39,0.000000
25%,10108525.000000,8.000000,1.000000,44.990000,2022-12-06 16:12:06,104.900000
50%,21274586.000000,17.000000,1.000000,79.980000,2023-02-22 11:04:22,210.400000
75%,28038562.000000,30.000000,1.000000,131.772500,2023-05-09 18:48:02,300.000000
max,46661804.000000,60.000000,1.000000,14499.710000,2023-08-01 23:57:46,978.700000
std,11389040.436254,15.270910,0.402613,107.438512,NaN,122.590689


Схлопнуть до одного чека для каждого клиента

In [7]:
df_sum_agg = df_train.groupby(ids_cols, as_index=True).agg({"revenue": ["sum", "count"]})
df_sum_agg.columns = ["revenue_sum", "items_count"]
df_sum_agg.reset_index(inplace=True)

In [8]:
df_times = df_sum_agg.groupby("customer_id", as_index=False).agg({"startdatetime": ["min", "max"]})
df_times.columns = ["customer_id", "startdatetime_min", "startdatetime_max"]

In [9]:
df_times['delta_days'] = (df_times['startdatetime_max'] - df_times['startdatetime_min']) / np.timedelta64(1, 's') / 60 / 60 /24

In [10]:
df_times.describe(percentiles=[i / 10 for i in range(1, 11)] + [0.95, 0.97, 0.99])

,customer_id,startdatetime_min,startdatetime_max,delta_days
count,500000.000000,500000,500000,500000.000000
mean,20246346.095608,2023-01-30 02:38:34.249616,2023-03-04 15:28:05.646236,33.534391
min,29891.000000,2022-09-02 00:00:39,2022-09-02 00:05:28,0.000231
10%,4015495.600000,2022-10-03 15:33:00.500000,2022-10-31 12:38:09.300000,9.972993
20%,8634336.400000,2022-11-03 14:13:11.800000,2022-11-30 14:45:51.400000,18.003245
30%,12046489.400000,2022-12-04 12:06:39.200000,2022-12-30 17:55:43,24.665043
40%,18298842.800000,2023-01-02 17:22:14.200000,2023-01-30 18:57:00.400000,30.053190
50%,21725643.000000,2023-02-01 18:15:01.500000,2023-02-28 17:46:36.500000,35.119381
60%,24118279.600000,2023-03-03 17:46:06.800000,2023-03-31 20:18:55.800000,40.101292
70%,26967157.300000,2023-04-03 13:34:39.900000,2023-04-30 19:09:52.400000,44.734309


Рассмотрим окна 7, 14, 28 дней

In [11]:
def calculate_window_features(dataset, ids_cols, windows):
    df_sum_agg = dataset.groupby(ids_cols, as_index=True).agg({"revenue": ["sum", "count"]})
    df_sum_agg.columns = ["revenue_sum", "items_count"]
    df_sum_agg.reset_index(inplace=True)
    
    df_max_dt = df_sum_agg.groupby(['customer_id'], as_index=False).agg({"startdatetime": ["max"]})
    df_max_dt.columns = ["customer_id", "max_startdatetime"]
    df_sum_agg = df_sum_agg.merge(df_max_dt, on="customer_id", how='inner')
    df_sum_agg['delta_days'] = (df_sum_agg['max_startdatetime'] - df_sum_agg['startdatetime']) / np.timedelta64(1, 's') / 60 / 60 / 24
    
    new_ids_cols = list(set(ids_cols).difference(["startdatetime"]))
    agg_dfs = []
    for window in windows:
        filtered_df = df_sum_agg[df_sum_agg['delta_days'] < 30]
        filtered_df = (
            filtered_df
            .groupby(new_ids_cols, as_index=False)
            .agg({
                "revenue_sum": ["max", "mean", "std", "sum", "count"], 
                "items_count": ["max", "mean", "std", "sum"], 
            })
        )
        filtered_df.columns = new_ids_cols + [
            f"{col}_{agg_func}_{window}d" for col in ["revenue_sum", "items_count"]
            for agg_func in ["max", "mean", "std", "sum", "count"]
        ][:-1]
        agg_dfs.append(filtered_df)
    ids_df = df_sum_agg[new_ids_cols].drop_duplicates().reset_index(drop=True)
    for temp_df in agg_dfs:
        ids_df = ids_df.merge(temp_df, on=new_ids_cols, how='left')
    return ids_df

In [12]:
windows = [7, 14, 28]

In [13]:
df_windows_agg = calculate_window_features(df_train, ['customer_id', 'buy_post', 'startdatetime'], windows)

In [14]:
df_windows_agg.describe()

,buy_post,customer_id,revenue_sum_max_7d,revenue_sum_mean_7d,revenue_sum_std_7d,revenue_sum_sum_7d,revenue_sum_count_7d,items_count_max_7d,items_count_mean_7d,items_count_std_7d,items_count_sum_7d,revenue_sum_max_14d,revenue_sum_mean_14d,revenue_sum_std_14d,revenue_sum_sum_14d,revenue_sum_count_14d,items_count_max_14d,items_count_mean_14d,items_count_std_14d,items_count_sum_14d,revenue_sum_max_28d,revenue_sum_mean_28d,revenue_sum_std_28d,revenue_sum_sum_28d,revenue_sum_count_28d,items_count_max_28d,items_count_mean_28d,items_count_std_28d,items_count_sum_28d
count,500000.000000,500000.000000,500000.000000,500000.000000,468552.000000,500000.000000,500000.000000,500000.000000,500000.000000,468552.000000,500000.000000,500000.000000,500000.000000,468552.000000,500000.000000,500000.000000,500000.000000,500000.000000,468552.000000,500000.000000,500000.000000,500000.000000,468552.000000,500000.000000,500000.000000,500000.000000,500000.000000,468552.000000,500000.000000
mean,0.718190,20246346.095608,757.652491,494.607167,247.689393,1845.637883,4.064160,6.644806,4.448378,2.082372,17.171574,757.652491,494.607167,247.689393,1845.637883,4.064160,6.644806,4.448378,2.082372,17.171574,757.652491,494.607167,247.689393,1845.637883,4.064160,6.644806,4.448378,2.082372,17.171574
std,0.449882,11496610.350691,495.770731,317.598032,219.980227,1483.067628,3.072057,3.767259,2.400114,1.700930,13.900050,495.770731,317.598032,219.980227,1483.067628,3.072057,3.767259,2.400114,1.700930,13.900050,495.770731,317.598032,219.980227,1483.067628,3.072057,3.767259,2.400114,1.700930,13.900050
min,0.000000,29891.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,0.000000,10731681.250000,414.000000,283.531894,103.635579,900.940000,3.000000,4.000000,2.750000,0.957427,8.000000,414.000000,283.531894,103.635579,900.940000,3.000000,4.000000,2.750000,0.957427,8.000000,414.000000,283.531894,103.635579,900.940000,3.000000,4.000000,2.750000,0.957427,8.000000
50%,1.000000,21725643.000000,639.960000,415.220000,191.360307,1474.920000,3.000000,6.000000,4.000000,1.732051,14.000000,639.960000,415.220000,191.360307,1474.920000,3.000000,6.000000,4.000000,1.732051,14.000000,639.960000,415.220000,191.360307,1474.920000,3.000000,6.000000,4.000000,1.732051,14.000000
75%,1.000000,28452387.000000,969.920000,619.968500,328.324037,2352.820000,5.000000,8.000000,5.666667,2.828427,22.000000,969.920000,619.968500,328.324037,2352.820000,5.000000,8.000000,5.666667,2.828427,22.000000,969.920000,619.968500,328.324037,2352.820000,5.000000,8.000000,5.666667,2.828427,22.000000
max,1.000000,46661804.000000,23294.220000,8485.035000,10584.037940,81094.770000,171.000000,190.000000,54.000000,93.510695,667.000000,23294.220000,8485.035000,10584.037940,81094.770000,171.000000,190.000000,54.000000,93.510695,667.000000,23294.220000,8485.035000,10584.037940,81094.770000,171.000000,190.000000,54.000000,93.510695,667.000000


In [15]:
df_windows_agg_test = calculate_window_features(df_test, ['customer_id', 'startdatetime'], windows)

In [16]:
df_windows_agg_test.describe()

,customer_id,revenue_sum_max_7d,revenue_sum_mean_7d,revenue_sum_std_7d,revenue_sum_sum_7d,revenue_sum_count_7d,items_count_max_7d,items_count_mean_7d,items_count_std_7d,items_count_sum_7d,revenue_sum_max_14d,revenue_sum_mean_14d,revenue_sum_std_14d,revenue_sum_sum_14d,revenue_sum_count_14d,items_count_max_14d,items_count_mean_14d,items_count_std_14d,items_count_sum_14d,revenue_sum_max_28d,revenue_sum_mean_28d,revenue_sum_std_28d,revenue_sum_sum_28d,revenue_sum_count_28d,items_count_max_28d,items_count_mean_28d,items_count_std_28d,items_count_sum_28d
count,112334.000000,112334.000000,112334.000000,105282.000000,112334.000000,112334.000000,112334.000000,112334.000000,105282.000000,112334.000000,112334.000000,112334.000000,105282.000000,112334.000000,112334.000000,112334.000000,112334.000000,105282.000000,112334.000000,112334.000000,112334.000000,105282.000000,112334.000000,112334.000000,112334.000000,112334.000000,105282.000000,112334.000000
mean,20748611.321951,758.455421,495.420288,252.123928,1780.666537,3.868366,6.565172,4.403679,2.084958,16.347491,758.455421,495.420288,252.123928,1780.666537,3.868366,6.565172,4.403679,2.084958,16.347491,758.455421,495.420288,252.123928,1780.666537,3.868366,6.565172,4.403679,2.084958,16.347491
std,11582578.505840,494.515099,316.901508,222.529589,1392.809678,2.725431,3.725996,2.365839,1.701388,12.968391,494.515099,316.901508,222.529589,1392.809678,2.725431,3.725996,2.365839,1.701388,12.968391,494.515099,316.901508,222.529589,1392.809678,2.725431,3.725996,2.365839,1.701388,12.968391
min,52341.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,11041231.250000,409.990000,283.310000,104.905484,879.952500,3.000000,4.000000,2.727273,0.957427,8.000000,409.990000,283.310000,104.905484,879.952500,3.000000,4.000000,2.727273,0.957427,8.000000,409.990000,283.310000,104.905484,879.952500,3.000000,4.000000,2.727273,0.957427,8.000000
50%,22155898.000000,639.950000,415.717500,194.858663,1434.920000,3.000000,6.000000,4.000000,1.732051,13.000000,639.950000,415.717500,194.858663,1434.920000,3.000000,6.000000,4.000000,1.732051,13.000000,639.950000,415.717500,194.858663,1434.920000,3.000000,6.000000,4.000000,1.732051,13.000000
75%,28861910.500000,969.970000,622.461875,334.176288,2270.855000,4.000000,8.000000,5.625000,2.828427,20.000000,969.970000,622.461875,334.176288,2270.855000,4.000000,8.000000,5.625000,2.828427,20.000000,969.970000,622.461875,334.176288,2270.855000,4.000000,8.000000,5.625000,2.828427,20.000000
max,46668221.000000,8099.100000,5296.370000,5023.081513,47242.440000,98.000000,92.000000,32.000000,51.384174,384.000000,8099.100000,5296.370000,5023.081513,47242.440000,98.000000,92.000000,32.000000,51.384174,384.000000,8099.100000,5296.370000,5023.081513,47242.440000,98.000000,92.000000,32.000000,51.384174,384.000000


### Information value

IV < 0,02 — отсутствует;

0,02 ≤ IV <0,1 — низкая;

0,1 ≤ IV < 0,3 — средняя;

IV ≥ 0,3 — высокая.

In [17]:
def calculate_information_value(dataset, feats, target_col):
    iv_by_feat = {}
    temp = dataset.copy()
    for feat in feats:
        temp["quant_range"] = pd.qcut(
            x=temp[feat], q=[0, 0.25, 0.5, 0.75, 1.0], duplicates="drop"
        )

        bins = {}
        for i, bin in enumerate(temp["quant_range"].unique()):
            bins[bin] = i

        temp["bin"] = temp["quant_range"].apply(lambda x: bins[x])

        iv = (
            pd.crosstab(temp["bin"], temp[target_col], normalize="columns")
            .assign(woe=lambda dfx: np.log(dfx[1] / dfx[0]))
            .assign(iv=lambda dfx: np.sum(dfx["woe"] * (dfx[1] - dfx[0])))
        )["iv"].unique()[0]

        iv_by_feat[feat] = [iv]
    df = pd.DataFrame(iv_by_feat).T
    df.reset_index(inplace=True)
    df.columns = ["feature", "IV"]
    return df

In [18]:
feats = list(set(df_windows_agg.columns).difference(ids_cols))

In [19]:
iv_df = calculate_information_value(df_windows_agg, feats, "buy_post")

In [20]:
iv_df.sort_values(by="IV", ascending=False)

,feature,IV
3,revenue_sum_count_7d,0.155249
26,revenue_sum_count_14d,0.155249
13,revenue_sum_count_28d,0.155249
12,items_count_sum_14d,0.088741
16,items_count_sum_28d,0.088741
8,items_count_sum_7d,0.088741
23,revenue_sum_sum_28d,0.043770
5,revenue_sum_sum_14d,0.043770
0,revenue_sum_sum_7d,0.043770
19,items_count_max_28d,0.012120
